In [44]:
import pandas as pd
import os

In [45]:
# Caminho para ler os dados brutos
data_raw_path = '../data_raw/'
# Caminho para salvar os dados processados
processed_data_path = '../data_processed/'
# Cria o diretório de dados processados se ele não existir
os.makedirs(processed_data_path, exist_ok=True)

Carregando Todos os DataFrames

In [46]:
try:
    path_desemprego = os.path.join(data_raw_path, 'desemprego_trimestral.csv')
    path_dre_empresas = os.path.join(data_raw_path, 'dre_anual_empresas.csv')
    path_gtrends = os.path.join(data_raw_path, 'gtrends.csv')
    path_icc = os.path.join(data_raw_path, 'icc_consumidor_mensal.csv')
    path_ipca = os.path.join(data_raw_path, 'ipca_mensal.csv')
    path_pib = os.path.join(data_raw_path, 'pib_mensal.csv')
    path_selic = os.path.join(data_raw_path, 'selic_meta.csv')

    df_desemprego = pd.read_csv(path_desemprego)
    df_empresas = pd.read_csv(path_dre_empresas)
    df_gtrends = pd.read_csv(path_gtrends)
    df_icc = pd.read_csv(path_icc)
    df_ipca = pd.read_csv(path_ipca)
    df_pib = pd.read_csv(path_pib)
    df_selic = pd.read_csv(path_selic)

except FileNotFoundError as e:
    print(f"🚨 ERRO: Arquivo não encontrado: {e.filename}")
except Exception as e:
    print(f"🚨 ERRO ao carregar DataFrames: {e}")

Transformação consolidada dos Dataframes

In [47]:
#-----------------------------------------------------------------------------
# Transformação df_desemprego
#-----------------------------------------------------------------------------
codigos_str = df_desemprego['periodo_codigo_trimestre'].astype(str) # Converte o int64 para string
anos = codigos_str.str[:4] # Extrai o ano (primeiros 4) e o trimestre (último)
trimestres_num = codigos_str.str[-1]  # Pega apenas o último dígito para o trimestre
periodos_q_str = anos + "Q" + trimestres_num # Cria uma string no formato "AAAAQT"
periodos_index = pd.PeriodIndex(periodos_q_str, freq='Q') # Converte essa string para um objeto PeriodIndex trimestral
datas_fim_trimestre = periodos_index.to_timestamp(how='end') # Converte o PeriodIndex para Timestamps no FIM do período
df_desemprego['data'] = datas_fim_trimestre # Cria a nova coluna 'data' e remove a antiga
df_desemprego = df_desemprego.drop(columns=['periodo_codigo_trimestre']) #Exclui a coluna antiga

#-----------------------------------------------------------------------------
# Transformação colunas 'data'
#-----------------------------------------------------------------------------

# Definir os DataFrames e seus formatos esperados
dataframes_para_converter = [
    {"nome": "Selic",        "df": df_selic,      "formato": "%d/%m/%Y"},
    {"nome": "IPCA",         "df": df_ipca,       "formato": "%d/%m/%Y"},
    {"nome": "PIB (IBC-Br)", "df": df_pib,        "formato": "%d/%m/%Y"},
    {"nome": "gtrends",      "df": df_gtrends,    "formato": "%Y-%m-%d"},
    {"nome": "ICC",          "df": df_icc,        "formato": "%d/%m/%Y"},
    {"nome": "DRE_empresas", "df": df_empresas,   "formato": "%Y-%m-%d"}
]

for item in dataframes_para_converter:
    nome_df = item["nome"]
    df = item["df"]
    formato_data = item["formato"]

    # Verifica se já não é datetime
    if not pd.api.types.is_datetime64_any_dtype(df['data']):
        try:
            # Converte usando o formato especificado
            df['data'] = pd.to_datetime(df['data'], format=formato_data)
            print(f"   > Coluna 'data' do DataFrame '{nome_df}' convertida.")
        except Exception as e:
            formato_tentado = formato_data if formato_data else "automático"
            print(f"   🚨 ERRO ao converter 'data' no DataFrame '{nome_df}' (formato tentado: {formato_tentado}): {e}")

   > Coluna 'data' do DataFrame 'Selic' convertida.
   > Coluna 'data' do DataFrame 'IPCA' convertida.
   > Coluna 'data' do DataFrame 'PIB (IBC-Br)' convertida.
   > Coluna 'data' do DataFrame 'gtrends' convertida.
   > Coluna 'data' do DataFrame 'ICC' convertida.
   > Coluna 'data' do DataFrame 'DRE_empresas' convertida.


Colocando colunas 'data' como indexadores

In [48]:
dfs_para_indexar = {
    "Selic": df_selic,
    "IPCA": df_ipca,
    "PIB (IBC-Br)": df_pib,
    "ICC": df_icc,
    "Desemprego": df_desemprego,
    "Gtrends": df_gtrends
}

for nome, df in dfs_para_indexar.items():
    if df is not None and 'data' in df.columns:
        try:
            df.set_index('data', inplace=True)
            df.sort_index(inplace=True)
            print(f"   > Índice definido e ordenado para DataFrame '{nome}'.")
        except KeyError:
             print(f"   ⚠️ AVISO: Coluna 'data' não encontrada em '{nome}' (talvez já seja índice?).")
        except Exception as e:
            print(f"   🚨 ERRO ao definir índice para '{nome}': {e}")


   > Índice definido e ordenado para DataFrame 'Selic'.
   > Índice definido e ordenado para DataFrame 'IPCA'.
   > Índice definido e ordenado para DataFrame 'PIB (IBC-Br)'.
   > Índice definido e ordenado para DataFrame 'ICC'.
   > Índice definido e ordenado para DataFrame 'Desemprego'.
   > Índice definido e ordenado para DataFrame 'Gtrends'.


Verificação dos Dataframes Finais (ajustados)

In [49]:
dataframes = {
    "Desemprego": df_desemprego,
    "DRE Anual empresas": df_empresas,
    "Google Trends": df_gtrends,
    "ICC Consumidor": df_icc,
    "IPCA Mensal": df_ipca,
    "PIB": df_pib,
    "Selic Meta": df_selic
}

files_list = os.listdir(data_raw_path)

for nome, df in dataframes.items():
    print(f"\nAnalisando {nome}")
    # Verifica se o DataFrame realmente existe (segurança)
    if df is None:
        print("   > DataFrame não carregado.")
        continue
    try:
        df.info()
        null_counts = df.isnull().sum()
        print(null_counts[null_counts > 0])
        if null_counts.sum() == 0:
            print("Nenhum valor ausente encontrado\n")
        
        print(df.head())
    
    except Exception as e:
        print(f"🚨 ERRO ao inspecionar o DataFrame {nome}: {e}")



Analisando Desemprego
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 22 entries, 2020-03-31 23:59:59.999999999 to 2025-06-30 23:59:59.999999999
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   taxa_desocupacao  22 non-null     float64
dtypes: float64(1)
memory usage: 352.0 bytes
Series([], dtype: int64)
Nenhum valor ausente encontrado

                               taxa_desocupacao
data                                           
2020-03-31 23:59:59.999999999              12.4
2020-06-30 23:59:59.999999999              13.6
2020-09-30 23:59:59.999999999              14.9
2020-12-31 23:59:59.999999999              14.2
2021-03-31 23:59:59.999999999              14.9

Analisando DRE Anual empresas
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 59 columns):
 #   Column                                                      Non-Null Count  Dtype         
---  --